## _Clustering_ pada Data GDELT

### Data

Data yang digunakan adalah data GoldsteinScale, NumMentions, NumSources dan NumArticles dari pemberitaan dengan tema yang sudah ditentukan sebelumnya.

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pymysql

In [2]:
gw = pd.read_excel('Data_Master.xlsx', 'Global Warming')
pa = pd.read_excel('Data_Master.xlsx', 'Penyebab Alami')
pi = pd.read_excel('Data_Master.xlsx', 'Perubahan Iklim')
mi = pd.read_excel('Data_Master.xlsx', 'Mitigasi')
pe = pd.read_excel('Data_Master.xlsx', 'Penyakit')

In [20]:
gwm = gw[['SQLDATE','GoldsteinScale','NumMentions','NumSources', 'NumArticles', 'AvgTone']]
pal = pa[['SQLDATE','GoldsteinScale','NumMentions','NumSources', 'NumArticles', 'AvgTone']]
pik = pi[['SQLDATE','GoldsteinScale','NumMentions','NumSources', 'NumArticles', 'AvgTone']]
mit = mi[['SQLDATE','GoldsteinScale','NumMentions','NumSources', 'NumArticles', 'AvgTone']]
pen = pe[['SQLDATE','GoldsteinScale','NumMentions','NumSources', 'NumArticles', 'AvgTone']]

In [37]:
print(gwm['GoldsteinScale'].mean(), pal['GoldsteinScale'].mean(), pik['GoldsteinScale'].mean(), mit['GoldsteinScale'].mean(), pen['GoldsteinScale'].mean())
print(gwm['NumMentions'].mean(), pal['NumMentions'].mean(), pik['NumMentions'].mean(), mit['NumMentions'].mean(), pen['NumMentions'].mean())
print(gwm['NumSources'].mean(), pal['NumSources'].mean(), pik['NumSources'].mean(), mit['NumSources'].mean(), pen['NumSources'].mean())
print(gwm['NumArticles'].mean(), pal['NumArticles'].mean(), pik['NumArticles'].mean(), mit['NumArticles'].mean(), pen['NumArticles'].mean())


1.7755469399058403 0.804686548995745 2.3891380826736914 1.945879518072272 1.1549261825851254
7.546109111049571 22.620511259890446 6.754617414248021 6.773493975903614 18.907200964145826
1.4608141788978122 4.3116250760803405 1.4067722075637643 1.3262650602409638 4.042783971075625
7.476876211575741 22.141509433962263 6.703605980650836 6.6891566265060245 18.67128653208798


In [44]:
data1 = pd.Series([1.154926,  0.804686548995745, 2.3891380826736914, 1.945879518072272, 1.1549261825851254],
                 index=['Global Warming', 'Penyebab Alami', 'Perubahan Iklim', 'Mitigasi', 'Penyakit'])
data2 = pd.Series([7.546109111049571, 22.620511259890446, 6.754617414248021, 6.773493975903614, 18.907200964145826],
                 index=['Global Warming', 'Penyebab Alami', 'Perubahan Iklim', 'Mitigasi', 'Penyakit'])
data3 = pd.Series([1.4608141788978122, 4.3116250760803405, 1.4067722075637643, 1.3262650602409638, 4.042783971075625],
                 index=['Global Warming', 'Penyebab Alami', 'Perubahan Iklim', 'Mitigasi', 'Penyakit'])
data4 = pd.Series([7.476876211575741, 22.141509433962263, 6.703605980650836, 6.6891566265060245, 18.67128653208798],
                 index=['Global Warming', 'Penyebab Alami', 'Perubahan Iklim', 'Mitigasi', 'Penyakit'])

In [47]:
data = pd.DataFrame({'GoldsteinScale': data1, 'NumMentions': data2, 'NumSources': data3, 'NumArticles': data4})

In [70]:
data.corr()

,GoldsteinScale,NumMentions,NumSources,NumArticles
GoldsteinScale,1.000000,-0.760014,-0.733637,-0.759648
NumMentions,-0.760014,1.000000,0.993597,0.999969
NumSources,-0.733637,0.993597,1.000000,0.994428
NumArticles,-0.759648,0.999969,0.994428,1.000000


Akan digunakan _clustering_ untuk mengetahui artikel manakah yang menjaadi _tending_ diantara artikel lainnya

Sebelum dilakukan _clustering_, harus ditentukan banyaknya _cluster_ yang ingin dibentuk terlebiih dahulu. Dalam penelitian ingin diketahui apakah artikel masuk _trending_ atau tidak, sehingga ditentukan banyaknya _cluster_ yaitu 2 _cluster_.

In [52]:
from sklearn import cluster

In [54]:
X = data
y = data.index
k_means = cluster.KMeans(n_clusters=2)
k_means.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [62]:
print(k_means.labels_[:])

[0 1 0 0 1]


In [66]:
dataclust = pd.DataFrame({'GoldsteinScale': data1, 'NumMentions': data2, 'NumSources': data3, 'NumArticles': data4, 'Clust': k_means.labels_[:]})

In [67]:
dataclust

,GoldsteinScale,NumMentions,NumSources,NumArticles,Clust
Global Warming,1.154926,7.546109,1.460814,7.476876,0
Penyebab Alami,0.804687,22.620511,4.311625,22.141509,1
Perubahan Iklim,2.389138,6.754617,1.406772,6.703606,0
Mitigasi,1.945880,6.773494,1.326265,6.689157,0
Penyakit,1.154926,18.907201,4.042784,18.671287,1


_Cluster_ 0 menyatakan bahwa artikel bukan _trending_ topik yang meliputi artikel dengan tema global warming, perubahan iklim, dan mitigasi. _Cluster_ 1 menyatakan bahwa bahwa artikel merupakan _trending_ topik yang meliputi artikel dengan tema penyebab alami dan penyakit.